<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import sys
import numpy as np
import pandas as pd


In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
def listyfy( corpus ): 
    return np.char.split(corpus)

def dictionarify(corpus): 
    dictionary = set()

    for document in corpus:
        dictionary.update(document)
    return dictionary


listed_corpus = listyfy( corpus )
dict_corpus = dictionarify(listed_corpus)



print(dict_corpus)
print("")
print(listed_corpus)


{'el', 'es', 'muchas', 'gracias', 'martes', 'de', 'dia', 'hoy', 'que'}

[list(['que', 'dia', 'es', 'hoy'])
 list(['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'])
 list(['martes', 'muchas', 'gracias'])]


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [5]:
def onehotify (listed_corpus=listed_corpus , dict_corpus=dict_corpus):
    output = np.zeros((corpus.shape[0],len(dict_corpus)))

    for j, document in enumerate(listed_corpus):
        for i, term in enumerate(dict_corpus) :        
            if (document.count(term)>0) : output[j,i]= 1
            else : output[j,i]= 0
    
    return output

#ejemplo

onehot_corpus = onehotify(listed_corpus,dict_corpus)

print(listed_corpus[2])
print("")
print(dict_corpus)
print("")
print(onehot_corpus)

['martes', 'muchas', 'gracias']

{'el', 'es', 'muchas', 'gracias', 'martes', 'de', 'dia', 'hoy', 'que'}

[[0. 1. 0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0.]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [6]:
def frecuencify (listed_corpus=listed_corpus , dict_corpus=dict_corpus):
    output = np.zeros((corpus.shape[0],len(dict_corpus)))

    for i, term in enumerate(dict_corpus) :
        for j, document in enumerate(listed_corpus):

            output[j,i]= document.count(term)
    
    return output

#ejemplo            
print(listyfy(corpus)[2])
print("")
print(dictionarify(listyfy(corpus)))
print("")
print(frecuencify( listyfy(corpus) , dictionarify(listyfy(corpus))))

['martes', 'muchas', 'gracias']

{'el', 'es', 'muchas', 'gracias', 'martes', 'de', 'dia', 'hoy', 'que'}

[[0. 1. 0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 2. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0.]]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [7]:


def find_idf( listed_corpus , dict_corpus ):
    idf_vector = np.zeros([len(dict_corpus),])

    one_hot_temp = onehotify(listed_corpus,dict_corpus)
    vsum_one_hot_temp = np.sum(one_hot_temp,axis=0)
    
    for idx in range(len(idf_vector)):
        idf_vector[idx] = np.log( listed_corpus.shape[0] / vsum_one_hot_temp[idx])

    return idf_vector

    

def get_TFIDF( listed_corpus , dict_corpus ):
    idf = find_idf(listed_corpus=listed_corpus , dict_corpus=dict_corpus)
    tf = frecuencify(listed_corpus=listed_corpus , dict_corpus=dict_corpus)

    TFIDF = tf * idf.T
    return TFIDF


print(listed_corpus)
df = pd.DataFrame(get_TFIDF( listed_corpus , dict_corpus ), columns = list(dict_corpus))

df.head()

[list(['que', 'dia', 'es', 'hoy'])
 list(['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'])
 list(['martes', 'muchas', 'gracias'])]


,el,es,muchas,gracias,martes,de,dia,hoy,que
0,0.000000,0.405465,0.000000,0.000000,0.000000,0.000000,0.405465,0.405465,1.098612
1,1.098612,0.405465,0.000000,0.000000,0.810930,1.098612,0.405465,0.405465,0.000000
2,0.000000,0.000000,1.098612,1.098612,0.405465,0.000000,0.000000,0.000000,0.000000


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [18]:
#Como la evaluacion del coseno necesita numeros y no texto, voy a usar alguna de las representaciones de word2 vec realizadas
# de manera random, voy a usar la tf-idf, porque escuche que se ha usado mucho. 

def how_similar(corpus , corpus_doc_idx):
    similarty_indexes = np.zeros([corpus.shape[0],])

    print(similarty_indexes.shape)

    listed_corpus = listyfy( corpus )
    dict_corpus = dictionarify(listed_corpus)
    
    TDFIDF_word2vec = get_TFIDF( listed_corpus , dict_corpus )

    for idx in range(len(TDFIDF_word2vec)):
        similarty_indexes[idx] = cosine_similarity(TDFIDF_word2vec[corpus_doc_idx,:],TDFIDF_word2vec[idx,:] ) 

    return TDFIDF_word2vec


a = how_similar(corpus=corpus, corpus_doc_idx=0)

(3,)
